In [1]:
import pandas as pd
import numpy as np

movieF=pd.read_csv('C:/Users/jawahar/Documents/Data science/Project/Untitled Folder/movies_F.csv')
movieS=pd.read_csv('C:/Users/jawahar/Documents/Data science/Project/Untitled Folder/movies_S.csv')

In [2]:
final_movie=pd.concat([movieF, movieS], ignore_index=True)

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\jawahar\Documents\Data science\Project\RecSys\RecSys\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
final_movie['plot'] = final_movie['plot'].fillna("")

In [5]:
final_movie['plot_embedding'] = final_movie['plot'].apply(lambda x: model.encode(x,convert_to_numpy=True))

c:\Users\jawahar\Documents\Data science\Project\RecSys\RecSys\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [6]:
final_movie.head()

,Unnamed: 0.1,Unnamed: 0,title,year,imdb_rating,metascore,votes,genre,runtime,certificates,img_url,plot,plot_embedding
0,0,0,Psycho,1960,8.5,97.0,758866,"['Drama', 'Horror', 'Mystery']",6540.0,R,https://m.media-amazon.com/images/M/MV5BYjZhMz...,A secretary on the run for embezzlement takes ...,"[-0.025498573, 0.046960022, -0.041405685, 0.06..."
1,1,1,The Apartment,1960,8.3,94.0,210681,"['Comedy', 'Drama', 'Romance']",7500.0,Approved,https://m.media-amazon.com/images/M/MV5BNDdhMz...,A Manhattan insurance clerk tries to rise in h...,"[0.012486427, 0.012657471, -0.027960727, -0.02..."
2,2,2,Spartacus,1960,7.9,87.0,148871,"['Adventure', 'Biography', 'Drama']",11820.0,PG-13,https://m.media-amazon.com/images/M/MV5BMTcyOT...,The slave Spartacus survives brutal training a...,"[-0.034061685, 0.029615097, -0.019052375, -0.0..."
3,3,3,The Magnificent Seven,1960,7.7,74.0,107555,"['Action', 'Adventure', 'Drama']",7680.0,Approved,https://m.media-amazon.com/images/M/MV5BMzYyNz...,Seven gunfighters are hired by Mexican peasant...,"[-0.032821186, 0.020097407, -0.10404044, 0.052..."
4,4,4,À bout de souffle,1960,7.7,96.0,91699,"['Crime', 'Drama']",5400.0,Not Rated,https://m.media-amazon.com/images/M/MV5BZGI5MW...,"A small-time crook, hunted by the authorities ...","[0.009688734, 0.06871374, -0.021408405, 0.0068..."


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
x=cosine_similarity(final_movie.iloc[0]['plot_embedding'].reshape(1, -1), final_movie.iloc[1]['plot_embedding'].reshape(1, -1))

In [16]:
x[0][0]

np.float32(0.22674529)

In [18]:
def hybrid_similarity(movie_a, movie_b):
    # Genre overlap score
    genre_overlap = len(set(movie_a['genre']) & set(movie_b['genre'])) / max(len(movie_a['genre']), 1)

    # Year distance penalty
    year_distance = abs(movie_a['year'] - movie_b['year'])
    year_score = max(0, 1 - year_distance / 10)  # decay if over 10 years

    # Certificate match
    certificate_score = 1 if movie_a['certificates'] == movie_b['certificates'] else 0

    # Metascore similarity (scaled to [0,1])
    meta_diff = abs(movie_a['metascore'] - movie_b['metascore']) / 100 if not np.isnan(movie_a['metascore']) and not np.isnan(movie_b['metascore']) else 1
    metascore_score = 1 - meta_diff

    # Popularity scaling (log to compress range)
    pop_score = min(np.log1p(movie_b['votes']) / np.log1p(movie_a['votes']), 1.0)

    # Plot similarity (cosine)
    plot_sim = cosine_similarity(
        movie_a['plot_embedding'].reshape(1, -1),
        movie_b['plot_embedding'].reshape(1, -1)
    )[0][0]

    # Weighted hybrid score
    final_score = (
        0.3 * plot_sim +
        0.2 * genre_overlap +
        0.1 * year_score +
        0.1 * certificate_score +
        0.1 * metascore_score +
        0.2 * pop_score
    )

    return final_score

In [21]:
hybrid_similarity(final_movie.iloc[0],final_movie.iloc[3])

np.float64(0.4647841921760384)

In [22]:
final_movie.to_csv('C:/Users/jawahar/Documents/Data science/Project/Untitled Folder/final_movie_with_Sent_embedding.csv', index=False)

In [27]:
final_movie.shape

(32500, 13)

In [28]:
def recommend_movies(df, title, top_n=5):
    target_movie = df[df['title'] == title].iloc[0]

    scores = []
    for idx, row in df.iterrows():
        if row['title'] == title:
            continue
        score = hybrid_similarity(target_movie, row)
        scores.append((row['title'], score))

    ranked = sorted(scores, key=lambda x: x[1], reverse=True)
    return ranked[:top_n]


In [29]:
recommend_movies(final_movie, 'The Dark Knight', top_n=5)

[('The Dark Knight Rises', np.float64(0.7032975960730578)),
 ('Batman Begins', np.float64(0.7006100155388766)),
 ('The Incredible Hulk', np.float64(0.6598712909015052)),
 ('The Bourne Ultimatum', np.float64(0.6518550915002945)),
 ('Star Wars: Episode III - Revenge of the Sith',
  np.float64(0.6464297344492258))]

In [32]:
final_movie[final_movie['votes']>5000].shape

(16102, 13)

In [33]:
recommend_movies(final_movie[final_movie['votes']>5000], 'The Dark Knight', top_n=5)

[('The Dark Knight Rises', np.float64(0.7032975960730578)),
 ('Batman Begins', np.float64(0.7006100155388766)),
 ('The Incredible Hulk', np.float64(0.6598712909015052)),
 ('The Bourne Ultimatum', np.float64(0.6518550915002945)),
 ('Star Wars: Episode III - Revenge of the Sith',
  np.float64(0.6464297344492258))]

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
genre=final_movie['genre'].tolist()

In [37]:
vect=CountVectorizer().fit(genre)

In [40]:
columns=vect.get_feature_names_out()
columns

array(['action', 'adventure', 'animation', 'biography', 'comedy', 'crime',
       'documentary', 'drama', 'family', 'fantasy', 'fi', 'history',
       'horror', 'music', 'musical', 'mystery', 'news', 'romance', 'sci',
       'sport', 'thriller', 'war', 'western'], dtype=object)

In [42]:
genre_vect=vect.transform(final_movie['genre'])
genre_df=pd.DataFrame(genre_vect.toarray(),columns=columns)


In [43]:
genre_df.head()

,action,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,...,music,musical,mystery,news,romance,sci,sport,thriller,war,western
0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
final_movie.reset_index(inplace=True)
movie_genre=pd.merge(final_movie,genre_df,how='outer',left_index=True,right_index=True)

In [49]:
movie_genre.drop(columns=['Unnamed: 0'],inplace=True)

In [50]:
movie_genre.head()

,index,title,year,imdb_rating,metascore,votes,genre,runtime,certificates,img_url,...,music,musical,mystery,news,romance,sci,sport,thriller,war,western
0,0,Psycho,1960,8.5,97.0,758866,"['Drama', 'Horror', 'Mystery']",6540.0,R,https://m.media-amazon.com/images/M/MV5BYjZhMz...,...,0,0,1,0,0,0,0,0,0,0
1,1,The Apartment,1960,8.3,94.0,210681,"['Comedy', 'Drama', 'Romance']",7500.0,Approved,https://m.media-amazon.com/images/M/MV5BNDdhMz...,...,0,0,0,0,1,0,0,0,0,0
2,2,Spartacus,1960,7.9,87.0,148871,"['Adventure', 'Biography', 'Drama']",11820.0,PG-13,https://m.media-amazon.com/images/M/MV5BMTcyOT...,...,0,0,0,0,0,0,0,0,0,0
3,3,The Magnificent Seven,1960,7.7,74.0,107555,"['Action', 'Adventure', 'Drama']",7680.0,Approved,https://m.media-amazon.com/images/M/MV5BMzYyNz...,...,0,0,0,0,0,0,0,0,0,0
4,4,À bout de souffle,1960,7.7,96.0,91699,"['Crime', 'Drama']",5400.0,Not Rated,https://m.media-amazon.com/images/M/MV5BZGI5MW...,...,0,0,0,0,0,0,0,0,0,0


array(['action', 'adventure', 'animation', 'biography', 'comedy', 'crime',
       'documentary', 'drama', 'family', 'fantasy', 'fi', 'history',
       'horror', 'music', 'musical', 'mystery', 'news', 'romance', 'sci',
       'sport', 'thriller', 'war', 'western'], dtype=object)